In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE 

from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping
from keras import optimizers
from bayes_opt import BayesianOptimization

from functools import partial




df = pd.read_csv('data.csv')
list = ['Unnamed: 32','id']
df.drop(list,axis = 1, inplace = True)

#Encoding categorical data values
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['diagnosis'] = labelencoder.fit_transform(df['diagnosis'])


from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df['radius_mean'] = ss.fit_transform(df['radius_mean'].values.reshape(-1,1))
df['texture_mean'] = ss.fit_transform(df['texture_mean'].values.reshape(-1,1))
df['perimeter_mean'] = ss.fit_transform(df['perimeter_mean'].values.reshape(-1,1))
df['area_mean'] = ss.fit_transform(df['area_mean'].values.reshape(-1,1))
df['perimeter_se'] = ss.fit_transform(df['perimeter_se'].values.reshape(-1,1))
df['area_se'] = ss.fit_transform(df['area_se'].values.reshape(-1,1))
df['radius_worst'] = ss.fit_transform(df['radius_worst'].values.reshape(-1,1))
df['texture_worst'] = ss.fit_transform(df['texture_worst'].values.reshape(-1,1))
df['perimeter_worst'] = ss.fit_transform(df['perimeter_worst'].values.reshape(-1,1))
df['area_worst'] = ss.fit_transform(df['area_worst'].values.reshape(-1,1))

df_1 = pd.DataFrame(df['diagnosis'])
df.drop(['diagnosis'], axis =1, inplace = True)

from sklearn.decomposition import PCA
pca = PCA()
df_pca = pd.DataFrame(pca.fit_transform(df), columns=df.columns)


X = df_pca
Y = df_1['diagnosis']



Using TensorFlow backend.


In [2]:
print("Shape of X: {}".format(X.shape))
print("Shape of Y: {}".format(Y.shape))

Shape of X: (569, 30)
Shape of Y: (569,)


In [3]:
#Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, 
                                                    stratify = Y)

sm = SMOTE(random_state=42, kind = 'borderline1')

X_resampled, Y_resampled = sm.fit_resample(X_train, Y_train)


from keras import optimizers

input_shape = (30,)

def get_model(input_shape, verbose, dropout2_rate, lr):
    """Builds a Sequential CNN model to recognize MNIST.

    Args:
      input_shape: Shape of the input depending on the `image_data_format`.
      dropout2_rate: float between 0 and 1. Fraction of the input units to drop for `dropout_2` layer.

    Returns:
      a Keras model

    """
    # Reset the tensorflow backend session.
    # tf.keras.backend.clear_session()
    # Define a CNN model to recognize MNIST.
    model = Sequential()
    model.add(Dense(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout2_rate, name="dropout_2"))
    model.add(Dense(1, activation='sigmoid'))


    
    # Train the model for a specified number of epochs.
    optimizer = optimizers.RMSprop(lr=lr)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # Train the model with the train dataset.
    model.fit(X_resampled, Y_resampled, epochs=1, steps_per_epoch=468, verbose=verbose)

    # Evaluate the model with the eval dataset.
    score = model.evaluate(X_test,Y_test, steps=10, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Return the accuracy.

    return score[1]

verbose = 1
fit_with_partial = partial(get_model, input_shape, verbose)


# Bounded region of parameter space
pbounds = {'dropout2_rate': (0.1, 0.5), 'lr': (1e-4, 1e-2)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=10,)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------
Epoch 1/1
468/468 [==============================] - 1s 2ms/step - loss: 0.0121 - acc: 0.9958
Test loss: 0.05198447406291962
Test accuracy: 0.9912280440330505
|  1        |  0.9912   |  0.2668   |  0.007231 |
Epoch 1/1
468/468 [==============================] - 1s 2ms/step - loss: 0.0202 - acc: 0.9935
Test loss: 0.06619055569171906
Test accuracy: 0.9912280440330505
|  2        |  0.9912   |  0.1      |  0.003093 |
Epoch 1/1
468/468 [==============================] - 1s 2ms/step - loss: 0.0624 - acc: 0.9800
Test loss: 0.030429236590862274
Test accuracy: 0.9912280440330505
|  3        |  0.9912   |  0.1587   |  0.001014 |
Epoch 1/1
468/468 [==============================] - 1s 2ms/step - loss: 0.0194 - acc: 0.9938
Test loss: 0.026515042409300804
Test accuracy: 0.9912280440330505
|  4        |  0.9912   |  0.1745   |  0.003521 |
Epoch 1/1
468/468 [==============================] - 1s 3ms/st